# Pricing Calculator Demo

In [1]:
import yfinance as yf

def get_forex_data(base_currency: str, target_currency: str, start_date: str, end_date: str):
    """
    Retrieve the exchange rate data between two currencies and name the DataFrame using the combination 
    of base_currency and target_currency.
    
    :param base_currency: Base currency (e.g., 'USD', 'EUR', 'JPY')
    :param target_currency: Target currency (e.g., 'USD', 'EUR', 'JPY')
    :param start_date: Start date (format: 'YYYY-MM-DD')
    :param end_date: End date (format: 'YYYY-MM-DD')
    :return: Dynamically named DataFrame, named as base_currency+target_currency
    """
    # The format of the forex symbol is 'BASECURRENCY=TARGETCURRENCY'
    forex_symbol = f"{base_currency}{target_currency}=X"
    
    # Use yfinance to download the forex rate data
    forex_data = yf.download(forex_symbol, start=start_date, end=end_date)
    
    if forex_data.empty:
        print(f"No data found for the {base_currency}/{target_currency} currency pair")
        return None
    else:
        # Dynamically create the DataFrame name
        df_name = f"{base_currency.lower()}{target_currency.lower()}"
        
        # Create a DataFrame using the global variable dictionary
        globals()[df_name] = forex_data
        
        # Return the name of the dynamically created DataFrame
        return df_name

# Example usage
base_currency = 'USD'
target_currency = 'JPY'
start_date = '2023-01-01'
end_date = '2100-12-31'

df_name = get_forex_data(base_currency, target_currency, start_date, end_date)

# If data is retrieved, print the DataFrame name and data
if df_name:
    print(f"DataFrame name: {df_name}")
    display(globals()[df_name])


[*********************100%%**********************]  1 of 1 completed
DataFrame name: usdjpy


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-01-02,130.845001,131.035004,130.621002,130.845001,130.845001,0
2023-01-03,130.794006,131.182007,129.533997,130.794006,130.794006,0
2023-01-04,131.248001,132.169998,129.964005,131.248001,131.248001,0
2023-01-05,132.246002,133.992004,131.729996,132.246002,132.246002,0
2023-01-06,133.307007,134.740997,132.186996,133.307007,133.307007,0
...,...,...,...,...,...,...
2024-12-03,149.507996,150.214996,148.651001,149.507996,149.507996,0
2024-12-04,149.647003,151.216995,149.537003,149.647003,149.647003,0
2024-12-05,150.567001,150.684006,149.664993,150.567001,150.567001,0


# Fetch Spot Yield Curve Data

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the list of years
years = ['2023', '2024']

# Initialize a list to store the data
all_data = []

# Iterate through each year to fetch the corresponding interest rate data
for year in years:
    url = f'https://home.treasury.gov/resource-center/data-chart-center/interest-rates/pages/xml?data=daily_treasury_yield_curve&field_tdr_date_value={year}'
    response = requests.get(url)
    
    # Parse the XML data
    soup = BeautifulSoup(response.content, 'xml')
    
    # Extract all <entry> tags
    entries = soup.find_all('entry')
    
    # Iterate over each <entry> tag, extract interest rate information, and convert it to float
    for entry in entries:
        date = entry.find('d:NEW_DATE').text[:10]  # Extract the date
        
        # Parse interest rates and attempt to convert to float, setting to NaN if conversion fails
        def parse_float(tag):
            try:
                return float(tag.text)
            except (AttributeError, ValueError):
                return None
        
        row = {
            'Date': date,
            '1 Mo': parse_float(entry.find('d:BC_1MONTH')),
            '2 Mo': parse_float(entry.find('d:BC_2MONTH')),
            '3 Mo': parse_float(entry.find('d:BC_3MONTH')),
            '4 Mo': parse_float(entry.find('d:BC_4MONTH')),
            '6 Mo': parse_float(entry.find('d:BC_6MONTH')),
            '1 Yr': parse_float(entry.find('d:BC_1YEAR')),
            '2 Yr': parse_float(entry.find('d:BC_2YEAR')),
            '3 Yr': parse_float(entry.find('d:BC_3YEAR')),
            '5 Yr': parse_float(entry.find('d:BC_5YEAR')),
            '7 Yr': parse_float(entry.find('d:BC_7YEAR')),
            '10 Yr': parse_float(entry.find('d:BC_10YEAR')),
            '20 Yr': parse_float(entry.find('d:BC_20YEAR')),
            '30 Yr': parse_float(entry.find('d:BC_30YEAR'))
        }
        all_data.append(row)

# Create a pandas DataFrame
YC = pd.DataFrame(all_data)

# Convert the Date column to datetime type and sort the DataFrame from oldest to newest
YC['Date'] = pd.to_datetime(YC['Date'])
YC = YC.sort_values(by='Date')

# Reset the index
YC = YC.reset_index(drop=True)

# Display the first few rows of the DataFrame
display(YC)


,Date,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
0,2023-01-03,4.17,4.42,4.53,4.70,4.77,4.72,4.40,4.18,3.94,3.89,3.79,4.06,3.88
1,2023-01-04,4.20,4.42,4.55,4.69,4.77,4.71,4.36,4.11,3.85,3.79,3.69,3.97,3.81
2,2023-01-05,4.30,4.55,4.66,4.75,4.81,4.78,4.45,4.18,3.90,3.82,3.71,3.96,3.78
3,2023-01-06,4.32,4.55,4.67,4.74,4.79,4.71,4.24,3.96,3.69,3.63,3.55,3.84,3.67
4,2023-01-09,4.37,4.58,4.70,4.74,4.83,4.69,4.19,3.93,3.66,3.60,3.53,3.83,3.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,2024-09-25,4.79,4.78,4.69,4.64,4.36,3.89,3.53,3.49,3.52,3.65,3.79,4.18,4.14
435,2024-09-26,4.90,4.87,4.68,4.65,4.38,3.96,3.60,3.54,3.55,3.65,3.79,4.17,4.12
436,2024-09-27,4.90,4.87,4.68,4.64,4.35,3.90,3.55,3.49,3.50,3.60,3.75,4.15,4.10
437,2024-09-30,4.93,4.87,4.73,4.65,4.38,3.98,3.66,3.58,3.58,3.67,3.81,4.19,4.14


# Fetch Floating Rate SOFR

In [2]:
import requests
import pandas as pd

# FRED API endpoint and parameters
api_key = '2aa460f03b045cef8efddca32ec26ea0'
url = f'https://api.stlouisfed.org/fred/series/observations?series_id=SOFR&api_key={api_key}&file_type=json&frequency=d&observation_start=2018-01-01&observation_end=2100-12-31'

# Send the request and fetch the data
response = requests.get(url)
data = response.json()['observations']

# Extract the date and SOFR values and store them in a DataFrame
sofr = pd.DataFrame(data)
sofr = sofr[['date', 'value']]
sofr.columns = ['Date', 'SOFR']

# Convert the 'SOFR' column to numeric and handle any errors (e.g., missing or invalid values)
sofr['SOFR'] = pd.to_numeric(sofr['SOFR'], errors='coerce')

# Convert the 'SOFR' values from percentage form to decimal form
sofr['SOFR'] = sofr['SOFR'] / 100

# Convert the Date column in the SOFR DataFrame to datetime format
sofr['Date'] = pd.to_datetime(sofr['Date'])

# Identify missing values in the SOFR column
missing_dates = sofr[sofr['SOFR'].isna()]['Date']
if not missing_dates.empty:
    print("Missing SOFR values for the following dates:")
    print(missing_dates)

# Perform linear interpolation on the SOFR column to fill missing values
sofr['SOFR'] = sofr['SOFR'].interpolate(method='linear')

# After interpolation, check if there are still any missing values
still_missing = sofr[sofr['SOFR'].isna()]['Date']
if not still_missing.empty:
    print("Still missing SOFR values after interpolation for the following dates:")
    print(still_missing)
else:
    print("All missing SOFR values have been filled.")

# Display the DataFrame
display(sofr)

Missing SOFR values for the following dates:
39     2018-05-28
66     2018-07-04
109    2018-09-03
134    2018-10-08
159    2018-11-12
          ...    
1632   2024-07-04
1674   2024-09-02
1704   2024-10-14
1724   2024-11-11
1737   2024-11-28
Name: Date, Length: 74, dtype: datetime64[ns]
All missing SOFR values have been filled.


,Date,SOFR
0,2018-04-03,0.0183
1,2018-04-04,0.0174
2,2018-04-05,0.0175
3,2018-04-06,0.0175
4,2018-04-09,0.0175
...,...,...
1738,2024-11-29,0.0459
1739,2024-12-02,0.0464
1740,2024-12-03,0.0464
1741,2024-12-04,0.0459


In [3]:
import requests
import zipfile
import io
import xml.etree.ElementTree as ET
import os
import pandas as pd
from datetime import datetime, timedelta

def download_and_extract_zip(base_url, date, email, download_folder='C:\\Users\\kybun\\Kneron'):
    """
    Download and extract a zip file containing OIS data for a specific date.
    
    :param base_url: The base URL to download the zip file.
    :param date: The date for which the OIS data should be downloaded (format: YYYYMMDD).
    :param email: The email required for downloading the data.
    :param download_folder: The folder where the zip file will be extracted.
    :return: The path of the extracted XML file or None if the download fails.
    """
    download_url = f"{base_url}_{date}.zip?email={email}" 
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)
    
    response = requests.get(download_url)
    
    if response.status_code == 200:
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            extracted_files = z.namelist()
            z.extractall(download_folder)
            print(f"File successfully extracted! Date: {date}")
            
            xml_file_path = None
            for file_name in extracted_files:
                full_path = os.path.join(download_folder, file_name)
                if file_name.endswith('.xml'):
                    renamed_file_path = os.path.join(download_folder, f"OIS_{date}.xml")
                    os.rename(full_path, renamed_file_path)
                    xml_file_path = renamed_file_path
                elif file_name == "ISDA Standard Rate Curves Disclaimer.txt":
                    delete_disclaimer_file(full_path)
            return xml_file_path
    else:
        print(f"Download failed with status code: {response.status_code}")
        return None

def parse_xml_file(xml_file_path, date):
    """
    Parse the XML file to extract the OIS curve data.
    
    :param xml_file_path: The path to the XML file.
    :param date: The date corresponding to the OIS data.
    :return: A DataFrame containing the curve data with tenor, rate, and date.
    """
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
    interest_rate_curve = root.find('interestRateCurve')
    curve_points = []
    for curve_point in interest_rate_curve.find('ois').findall('curvepoint'):
        tenor = curve_point.find('tenor').text
        par_rate = float(curve_point.find('parrate').text)
        curve_points.append({'Tenor': tenor, 'Rate': par_rate, 'Date': date})
    df = pd.DataFrame(curve_points)
    return df

def delete_xml_file(file_path):
    """
    Delete the XML file that matches specific conditions.
    
    :param file_path: The path to the XML file to be deleted.
    """
    if os.path.exists(file_path) and file_path.endswith('.xml') and "OIS_" in os.path.basename(file_path):
        os.remove(file_path)
        print(f"Deleted file: {file_path}")
    else:
        print(f"File not deleted: {file_path} does not meet deletion criteria.")

def delete_disclaimer_file(file_path):
    """
    Delete the ISDA disclaimer file if it exists.
    
    :param file_path: The path to the disclaimer file to be deleted.
    """
    if os.path.exists(file_path) and file_path.endswith("ISDA Standard Rate Curves Disclaimer.txt"):
        os.remove(file_path)
        print(f"Deleted disclaimer file: {file_path}")
    else:
        print(f"File not deleted: {file_path} does not meet deletion criteria.")

def fetch_and_merge_ois_data(base_url, email, start_date, download_folder='C:\\Users\\kybun\\Kneron'):
    """
    Fetch OIS data for a range of dates, download and extract the zip files, and merge the data into a DataFrame.
    
    :param base_url: The base URL to download the zip file.
    :param email: The email required for downloading the data.
    :param start_date: The starting date for the OIS data (format: YYYY-MM-DD).
    :param download_folder: The folder where the zip files will be extracted.
    :return: A DataFrame containing the OIS data for all the requested dates.
    """
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.now()
    all_data_df = pd.DataFrame()

    current_date = start_date
    while current_date <= end_date:
        date_str = current_date.strftime('%Y%m%d')
        xml_file_path = download_and_extract_zip(base_url, date_str, email, download_folder)
        
        if xml_file_path:
            daily_df = parse_xml_file(xml_file_path, date_str)
            all_data_df = pd.concat([all_data_df, daily_df], ignore_index=True)
            delete_xml_file(xml_file_path)
        
        current_date += timedelta(days=1)

    all_data_df['Date'] = pd.to_datetime(all_data_df['Date'])
    ois_pivot_df = all_data_df.pivot(index='Date', columns='Tenor', values='Rate').reset_index()
    
    tenor_order = ['1M', '2M', '3M', '6M', '1Y', '2Y', '3Y', '5Y', '7Y', '10Y', '15Y', '20Y', '25Y', '30Y']
    ois_pivot_df = ois_pivot_df[['Date'] + [tenor for tenor in tenor_order if tenor in ois_pivot_df.columns]]

    return ois_pivot_df

def update_ois_data():
    """
    Update the existing OIS data file with new data.
    
    If the file exists, new data will be downloaded starting from the latest date. 
    If the file does not exist, all data from the start date will be downloaded.
    """
    base_url = "https://rfr.ihsmarkit.com/InterestRates_USD"
    email = "ganyangsanyang@gmail.com"
    data_file = "ois_data.csv"

    if os.path.exists(data_file):
        existing_data = pd.read_csv(data_file)
        existing_data['Date'] = pd.to_datetime(existing_data['Date'])
        latest_date = existing_data['Date'].max().strftime('%Y-%m-%d')
        print(f"The latest date in the existing data is: {latest_date}")
        
        # Start from the day after the latest date
        start_date = (datetime.strptime(latest_date, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')
        new_data = fetch_and_merge_ois_data(base_url, email, start_date)
        
        if not new_data.empty:
            updated_data = pd.concat([existing_data, new_data], ignore_index=True)
            updated_data.to_csv(data_file, index=False)
            print("OIS data has been updated and saved to the file: ois_data.csv")
        else:
            print("No new data to update.")
    else:
        print("No existing data file found. Downloading from the beginning.")
        start_date_str = "20230801"
        ois = fetch_and_merge_ois_data(base_url, email, start_date_str)
        ois.to_csv(data_file, index=False)
        print("OIS data has been saved to the file: ois_data.csv")

if __name__ == "__main__":
    update_ois_data()


The latest date in the existing data is: 2024-11-29
Download failed with status code: 404
Download failed with status code: 404
File successfully extracted! Date: 20241202
Deleted disclaimer file: C:\Users\kybun\Kneron\ISDA Standard Rate Curves Disclaimer.txt
Deleted file: C:\Users\kybun\Kneron\OIS_20241202.xml
File successfully extracted! Date: 20241203
Deleted disclaimer file: C:\Users\kybun\Kneron\ISDA Standard Rate Curves Disclaimer.txt
Deleted file: C:\Users\kybun\Kneron\OIS_20241203.xml
File successfully extracted! Date: 20241204
Deleted disclaimer file: C:\Users\kybun\Kneron\ISDA Standard Rate Curves Disclaimer.txt
Deleted file: C:\Users\kybun\Kneron\OIS_20241204.xml
File successfully extracted! Date: 20241205
Deleted disclaimer file: C:\Users\kybun\Kneron\ISDA Standard Rate Curves Disclaimer.txt
Deleted file: C:\Users\kybun\Kneron\OIS_20241205.xml
File successfully extracted! Date: 20241206
Deleted disclaimer file: C:\Users\kybun\Kneron\ISDA Standard Rate Curves Disclaimer.txt


In [4]:
ois_df=pd.read_csv('ois_data.csv')
ois_df['Date'] = pd.to_datetime(ois_df['Date'], format='%Y-%m-%d')
display(ois_df)

,Date,1M,2M,3M,6M,1Y,2Y,3Y,5Y,7Y,10Y,15Y,20Y,25Y,30Y
0,2023-08-01,0.053160,0.053370,0.053640,0.054330,0.053768,0.048179,0.044178,0.040177,0.038627,0.037668,0.037308,0.036678,0.035538,0.034429
1,2023-08-02,0.053160,0.053380,0.053690,0.054300,0.053520,0.047730,0.043760,0.040080,0.038670,0.037869,0.037670,0.037130,0.036030,0.034971
2,2023-08-03,0.053150,0.053440,0.053700,0.054350,0.053599,0.047889,0.044179,0.040867,0.039707,0.039098,0.039048,0.038568,0.037479,0.036449
3,2023-08-04,0.053131,0.053371,0.053610,0.054180,0.053129,0.046948,0.042978,0.039378,0.038198,0.037668,0.037749,0.037349,0.036329,0.035340
4,2023-08-07,0.053121,0.053371,0.053631,0.054221,0.053141,0.046821,0.042911,0.039480,0.038400,0.037960,0.038121,0.037761,0.036790,0.035830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,2024-12-02,0.045206,0.044778,0.044507,0.043697,0.042300,0.040041,0.038814,0.037685,0.037310,0.037208,0.037528,0.037358,0.036587,0.035683
350,2024-12-03,0.045194,0.044758,0.044482,0.043513,0.042032,0.039915,0.038878,0.037800,0.037530,0.037560,0.037960,0.037800,0.037040,0.036150
351,2024-12-04,0.045045,0.044610,0.044340,0.043311,0.041631,0.039404,0.038428,0.037453,0.037085,0.037130,0.037490,0.037315,0.036470,0.035590
352,2024-12-05,0.044908,0.044507,0.044278,0.043285,0.041713,0.039567,0.038499,0.037403,0.037041,0.036983,0.037343,0.037153,0.036368,0.035450


# Interest Swap(Fix for Flt) Valuation

## Int Swap Valuation Function

In the valuation process of an interest rate swap (IRS), we first calculate the Net Present Value (NPV) of the swap using the following formula:

$$
\text{NPV} = \sum_{i=1}^{n} \left( \frac{C_i^{\text{fixed}}}{(1 + r_i^{\text{fixed}})^t} \right) - \sum_{j=1}^{m} \left( \frac{C_j^{\text{floating}}}{(1 + r_j^{\text{floating}})^t} \right)
$$

Where:  
- \( C_i^{\text{fixed}} \): Fixed cash flow at each payment date  
- \( r_i^{\text{fixed}} \): Fixed interest rate  
- \( t \): Time at which the cash flow occurs  
- \( C_j^{\text{floating}} \): Floating cash flow at each payment date  
- \( r_j^{\text{floating}} \): Floating interest rate  

---

### Yield Curve Construction

Next, we construct the yield curve using piecewise linear interpolation to connect the spot rates observed in the market. The formula for interpolation is as follows:

$$
r(t) = r_i + \frac{(r_{i+1} - r_i)}{t_{i+1} - t_i} \times (t - t_i)
$$

Where:  
- \( r(t) \): Interpolated rate at time \( t \)  
- \( r_i \) and \( r_{i+1} \): Known rates at adjacent time points \( t_i \) and \( t_{i+1} \)  
- \( t \): Target time for which the rate is being interpolated  

---

### Fixed Leg Cash Flow Calculation

The cash flow for the fixed leg is calculated using the following formula:

$$
C_{\text{fixed}} = \text{notional} \times \text{fixed\_rate} \times \text{day\_count\_fraction}
$$

Where:  
- \(\text{notional}\): Notional amount of the swap  
- \(\text{fixed\_rate}\): Fixed interest rate  
- \(\text{day\_count\_fraction}\): Day count fraction for the payment period  

---

### Floating Leg Cash Flow Calculation

The cash flow for the floating leg is calculated using the following formula, which accounts for the SOFR rate and spread:

$$
C_{\text{float}} = \text{notional} \times (\text{SOFR} + \text{spread}) \times \text{day\_count\_fraction}
$$

Where:  
- \(\text{notional}\): Notional amount of the swap  
- \(\text{SOFR}\): Floating rate (SOFR rate)  
- \(\text{spread}\): Spread applied to the SOFR rate  
- \(\text{day\_count\_fraction}\): Day count fraction for the payment period  

---

### Discounted Cash Flow Calculation

To calculate the present value of cash flows, we discount future cash flows to the valuation date using the following formula:

$$
PV = \sum \left( \frac{C_t}{(1 + r_t)^t} \right)
$$

Where:  
- \( C_t \): Cash flow at time \( t \)  
- \( r_t \): Discount rate at time \( t \)  
- \( t \): Time at which the cash flow occurs  

---

### Fair Rate Calculation

The fair rate of an interest rate swap is the fixed rate that makes the present value of the fixed and floating cash flows equal. It is calculated using the following formula:

$$
\text{Fair Rate} = \frac{\sum_{i=1}^{n} \left( \frac{1}{(1 + r_i^{\text{floating}})^t} \right)}{\sum_{i=1}^{n} \left( \frac{1}{(1 + r_i^{\text{fixed}})^t} \right)}
$$

Where:  
- \( r_i^{\text{floating}} \): Discount rates for the floating leg cash flows  
- \( r_i^{\text{fixed}} \): Discount rates for the fixed leg cash flows  

---

### Initial Valuation

When entering into an interest rate swap, the net present value (NPV) is typically set to zero, meaning the value of the fixed leg is equal to the value of the floating leg. This ensures that the fixed rate (fair rate) is determined in such a way that neither party has an initial advantage.


In [6]:
import QuantLib as ql
import pandas as pd

def calculate_swap_valuation(start_date, end_date, valuation_date, fixed_rate, notional, payment_frequency, spread, ois_df, 
                             sofr_df,  # Add DataFrame containing SOFR historical fixing data
                             fixed_day_count=ql.Thirty360(ql.Thirty360.ISDA), floating_day_count=ql.Actual360(), 
                             business_convention=ql.ModifiedFollowing, currency=ql.USDCurrency()):
    
    # Set calendar
    calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)

    # Date adjustment function to align with business day
    def adjust_to_business_day(input_date, calendar, business_convention):
        # Convert pandas.Timestamp to QuantLib.Date
        if isinstance(input_date, pd.Timestamp):
            ql_date = ql.Date(input_date.day, input_date.month, input_date.year)
        else:
            ql_date = input_date
        
        if not calendar.isBusinessDay(ql_date):
            adjusted_date = calendar.adjust(ql_date, business_convention)
            print(f"The input date {ql_date} is not a valid business day, adjusted to {adjusted_date}")
            return adjusted_date
        return ql_date

    # Convert pandas.Timestamp to QuantLib.Date
    valuation_date = adjust_to_business_day(valuation_date, calendar, business_convention)
    start_date = adjust_to_business_day(start_date, calendar, business_convention)
    end_date = adjust_to_business_day(end_date, calendar, business_convention)

    # Set evaluation date
    ql.Settings.instance().evaluationDate = valuation_date

    # Create SOFR benchmark index
    sofr_index = ql.Sofr(ql.YieldTermStructureHandle())
    sofr_index.clearFixings()

    # Check if the valuation date is after the start date, if so, add historical fixing
    if valuation_date > start_date:
        print(f"Adding fixings for dates between {start_date} and {valuation_date}")
        # Extract SOFR fixing data after the contract start date
        relevant_sofr = sofr_df[(sofr_df['Date'] >= pd.to_datetime(start_date.ISO())) & (sofr_df['Date'] <= pd.to_datetime(valuation_date.ISO()))]

        # Add fixings only for business days
        fixing_dates = []
        fixing_values = []
        for _, row in relevant_sofr.iterrows():
            fixing_date = ql.Date(row['Date'].day, row['Date'].month, row['Date'].year)
            if calendar.isBusinessDay(fixing_date):
                fixing_dates.append(fixing_date)
                fixing_values.append(row['SOFR'])

        if fixing_dates:
            sofr_index.addFixings(fixing_dates, fixing_values)
        else:
            print("No valid fixing dates found between the start date and valuation date.")

    # Create fixed leg payment schedule
    fixed_schedule = ql.Schedule(start_date, end_date, ql.Period(payment_frequency),
                                 calendar, business_convention, business_convention, 
                                 ql.DateGeneration.Forward, False)

    # Create floating leg payment schedule
    floating_schedule = ql.Schedule(start_date, end_date, ql.Period(payment_frequency),
                                    calendar, business_convention, business_convention, 
                                    ql.DateGeneration.Forward, False)

    # Use fixed notional amount
    nominal = notional

    # Create a list of OISRateHelper using OIS market data from the DataFrame
    helpers = []
    for tenor in ois_df.columns[1:]:  # Skip 'Date' column
        rate = ois_df.loc[ois_df['Date'] == pd.to_datetime(valuation_date.ISO()), tenor].values[0]
        period = ql.Period(tenor)
        rate_quote = ql.QuoteHandle(ql.SimpleQuote(rate))  # Use rate data directly as a percentage
        ois_helper = ql.OISRateHelper(2, period, rate_quote, ql.Sofr())
        helpers.append(ois_helper)
    
    # Build a SOFR-based discount curve using PiecewiseLogLinearDiscount
    yield_curve = ql.PiecewiseLogLinearDiscount(valuation_date, helpers, fixed_day_count)
    yield_curve.enableExtrapolation()

    # Create YieldTermStructureHandle
    discount_curve = ql.YieldTermStructureHandle(yield_curve)

    # Create floating rate cash flows based on SOFR
    sofr_index = ql.Sofr(discount_curve)

    # Create the interest rate swap
    swap = ql.OvernightIndexedSwap(
        ql.OvernightIndexedSwap.Payer,                # Payer swap type
        nominal,                                      # Notional amount
        fixed_schedule,                               # Fixed leg payment schedule
        fixed_rate,                                   # Fixed interest rate
        fixed_day_count,                              # Day count convention for the fixed leg
        sofr_index,                                   # SOFR-based floating leg
        spread,                                       # Spread for the floating leg
        0,                                            # Payment delay
        business_convention                           # Payment adjustment convention
    )

    # Set discounting engine to discount cash flows to valuation date
    discounting_engine = ql.DiscountingSwapEngine(discount_curve)
    swap.setPricingEngine(discounting_engine)

    # Calculate NPV and fair rate
    npv = swap.NPV()
    fair_rate = swap.fairRate()

    # Calculate DV01 (Shift SOFR curve by 1 basis point)
    shift = 0.0001  # 1 basis point
    sofr_curve_shifted = ql.ZeroSpreadedTermStructure(discount_curve, ql.QuoteHandle(ql.SimpleQuote(shift)))
    sofr_handle_shifted = ql.YieldTermStructureHandle(sofr_curve_shifted)
    
    # Create interest rate swap using the shifted SOFR curve
    swap_shifted = ql.OvernightIndexedSwap(
        ql.OvernightIndexedSwap.Payer, 
        nominal, 
        fixed_schedule, 
        fixed_rate, 
        fixed_day_count, 
        ql.Sofr(sofr_handle_shifted), 
        spread,
        0,
        business_convention
    )
    swap_shifted.setPricingEngine(ql.DiscountingSwapEngine(sofr_handle_shifted))
    
    dv01 = abs(swap.NPV() - swap_shifted.NPV())

    # Generate cash flow table for the fixed leg
    fixed_leg_data = []
    for cf in swap.fixedLeg():
        coupon = ql.as_fixed_rate_coupon(cf)
        fixed_leg_data.append((coupon.date(), coupon.rate(), coupon.accrualPeriod(), coupon.amount()))
    fixed_leg_df = pd.DataFrame(fixed_leg_data, columns=["date", "rate", "tenor", "amount"]).style.format(
        {"amount": "{:.2f}", "rate": "{:.2%}"})

    # Generate cash flow table for the floating leg
    floating_leg_data = []
    for cf in swap.overnightLeg():
        coupon = ql.as_floating_rate_coupon(cf)
        floating_leg_data.append((coupon.date(), coupon.rate(), coupon.accrualPeriod(), coupon.amount()))
    floating_leg_df = pd.DataFrame(floating_leg_data, columns=["date", "rate", "tenor", "amount"]).style.format(
        {"amount": "{:.2f}", "rate": "{:.2%}"})

    return round(npv, 6), round(fair_rate, 6), round(dv01, 6), fixed_leg_df, floating_leg_df

# Test call
npv, fair_rate, dv01, fixed_leg_df, floating_leg_df = calculate_swap_valuation(
    start_date=pd.Timestamp('2024-05-17'),
    end_date=pd.Timestamp('2054-05-17'),
    valuation_date=pd.Timestamp('2024-09-16'),  # Valuation date set within the contract period
    fixed_rate=0.029115,
    notional=75000000,
    payment_frequency=ql.Semiannual,
    spread=0.0,
    ois_df=ois_df,    # Pass OIS data
    sofr_df=sofr      # Pass SOFR data
)

print(f"Swap Value: {npv}")
print(f"Fair Swap Rate: {fair_rate}")
print(f"DV01: {dv01}")
# Display cash flow tables
print("Fixed Leg Cash Flows:")
display(fixed_leg_df)
print("Floating Leg Cash Flows:")
display(floating_leg_df)


The input date May 17th, 2054 is not a valid business day, adjusted to May 18th, 2054
Adding fixings for dates between May 17th, 2024 and September 16th, 2024
Swap Value: 4005291.352129
Fair Swap Rate: 0.031875
DV01: 142278.203928
Fixed Leg Cash Flows:


,date,rate,tenor,amount
0,"November 18th, 2024",2.91%,0.502778,1097878.13
1,"May 19th, 2025",2.91%,0.502778,1097878.13
2,"November 17th, 2025",2.91%,0.494444,1079681.25
3,"May 18th, 2026",2.91%,0.502778,1097878.13
4,"November 17th, 2026",2.91%,0.497222,1085746.87
5,"May 17th, 2027",2.91%,0.500000,1091812.50
6,"November 17th, 2027",2.91%,0.500000,1091812.50
7,"May 17th, 2028",2.91%,0.500000,1091812.50
8,"November 17th, 2028",2.91%,0.500000,1091812.50
9,"May 17th, 2029",2.91%,0.500000,1091812.50


Floating Leg Cash Flows:


,date,rate,tenor,amount
0,"November 18th, 2024",5.26%,0.513889,2025370.80
1,"May 19th, 2025",3.84%,0.505556,1457268.47
2,"November 17th, 2025",3.05%,0.505556,1158067.40
3,"May 18th, 2026",2.85%,0.505556,1082203.14
4,"November 17th, 2026",2.81%,0.508333,1072141.99
5,"May 17th, 2027",2.87%,0.502778,1082228.25
6,"November 17th, 2027",2.85%,0.511111,1094008.73
7,"May 17th, 2028",2.95%,0.505556,1119435.99
8,"November 17th, 2028",2.92%,0.511111,1119435.99
9,"May 17th, 2029",2.97%,0.502778,1119435.99


# Int Cap/Floor Valuation

在Cap和Floor的估值过程中，首先需要设置估值日期，并确保该日期为有效的工作日。接着，我们从即期利率曲线（YC）中提取当日的市场利率，构建分段线性插值的贴现曲线，用于折现未来的现金流。分段线性插值公式如下：

$$
r(t) = r_i + \frac{(r_{i+1} - r_i)}{t_{i+1} - t_i} \times (t - t_i)
$$

其中，\( r(t) \) 是时间 \( t \) 对应的插值利率，\( r_i \) 和 \( r_{i+1} \) 是已知期限的利率，\( t_i \) 和 \( t_{i+1} \) 是相应的到期时间。

SOFR作为浮动利率的基准，通过历史数据的处理，SOFR指数会记录每个有效交易日的历史利率。这些数据用于计算浮动利率部分的现金流，计算公式如下：

$$
C_{\text{float}} = \text{notional} \times (\text{SOFR} + \text{spread}) \times \text{day\_count\_fraction}
$$

Cap和Floor是基于支付时间表的金融衍生品，时间表由起始日期、结束日期和支付频率决定。支付频率通常是按季度支付。

为了估算波动率，使用SOFR数据的历史值计算其标准差，波动率公式为：

$$
\sigma = \sqrt{\frac{1}{n-1} \sum_{i=1}^{n} \left( r_i - \bar{r} \right)^2 }
$$

这里，\( \sigma \) 是波动率，\( r_i \) 是SOFR的历史值，\( \bar{r} \) 是平均利率，\( n \) 是样本数量。波动率反映了利率的变化幅度。

最后，我们使用贴现曲线和波动率，应用 `BlackCapFloorEngine` 对Cap和Floor进行定价。定价时使用如下净现值（NPV）公式：

$$
\text{NPV} = \sum_{i=1}^{n} \frac{C_i}{(1 + r_i)^t}
$$

其中，\( C_i \) 是未来的现金流，\( r_i \) 是折现率，\( t \) 是时间。

Cap和Floor的现值取决于贴现后的现金流值和波动率。如果波动率较高，Cap和Floor的现值也可能更高。


In [12]:
import requests
import pandas as pd

# FRED API endpoint and parameters
api_key = '2aa460f03b045cef8efddca32ec26ea0'
url = f'https://api.stlouisfed.org/fred/series/observations?series_id=SOFR&api_key={api_key}&file_type=json&frequency=d&observation_start=2018-01-01&observation_end=2100-12-31'

# Send the request and fetch the data
response = requests.get(url)
data = response.json()['observations']

# Extract the date and SOFR values and store them in a DataFrame
sofr_df = pd.DataFrame(data)
sofr_df = sofr_df[['date', 'value']]
sofr_df.columns = ['Date', 'SOFR']

# Convert the 'SOFR' column to numeric and handle any errors (e.g., missing or invalid values)
sofr_df['SOFR'] = pd.to_numeric(sofr_df['SOFR'], errors='coerce')

# Convert the 'SOFR' values from percentage form to decimal form
sofr_df['SOFR'] = sofr_df['SOFR'] / 100

# Convert the Date column in the SOFR DataFrame to datetime format
sofr_df['Date'] = pd.to_datetime(sofr_df['Date'])

# Identify missing values in the SOFR column
missing_dates = sofr_df[sofr_df['SOFR'].isna()]['Date']
if not missing_dates.empty:
    print("Missing SOFR values for the following dates:")
    print(missing_dates)

# Perform linear interpolation on the SOFR column to fill missing values
sofr_df['SOFR'] = sofr_df['SOFR'].interpolate(method='linear')

# After interpolation, check if there are still any missing values
still_missing = sofr_df[sofr_df['SOFR'].isna()]['Date']
if not still_missing.empty:
    print("Still missing SOFR values after interpolation for the following dates:")
    print(still_missing)
else:
    print("All missing SOFR values have been filled.")

# Display the DataFrame
display(sofr_df)

Missing SOFR values for the following dates:
39     2018-05-28
66     2018-07-04
109    2018-09-03
134    2018-10-08
159    2018-11-12
          ...    
1632   2024-07-04
1674   2024-09-02
1704   2024-10-14
1724   2024-11-11
1737   2024-11-28
Name: Date, Length: 74, dtype: datetime64[ns]
All missing SOFR values have been filled.


,Date,SOFR
0,2018-04-03,0.0183
1,2018-04-04,0.0174
2,2018-04-05,0.0175
3,2018-04-06,0.0175
4,2018-04-09,0.0175
...,...,...
1738,2024-11-29,0.0459
1739,2024-12-02,0.0464
1740,2024-12-03,0.0464
1741,2024-12-04,0.0459


In [14]:
import pandas as pd

# Set the file path
file_path = "OIS_Curve12.29.2023.csv"

# Read the CSV file
ois_df = pd.read_csv(file_path)

# Display the first few rows of the data to verify the import is correct
print(ois_df.head())

# Read the volatility CSV file
vola_df = pd.read_csv('Volatility.csv')

# Display the volatility DataFrame
display(vola_df)


   Term  Market Rate  Shift  Shifted Rate  Zero Rate  Discount
0  1 WK       5.3405      0        5.3405    5.41186  0.998222
1  2 WK       5.3431      0        5.3431    5.41174  0.997187
2  3 WK       5.3460      0        5.3460    5.41183  0.996152
3  1 MO       5.3480      0        5.3480    5.40940  0.994384
4  2 MO       5.3455      0        5.3455    5.39660  0.990289


,Tenor,0.00%,1.00%,1.50%,2.00%,2.50%,3.00%,3.50%,4.00%,5.00%,6.00%,7.00%,8.00%,9.00%,10.00%
0,1Yr,187.52,170.81,161.64,152.00,141.94,131.57,121.11,110.93,91.93,93.94,116.27,135.26,153.45,171.28
1,2Yr,156.36,147.54,143.02,138.61,134.43,130.53,126.78,122.90,114.07,124.37,147.62,166.30,183.15,199.25
2,3Yr,141.47,135.41,132.51,129.92,127.80,126.23,125.10,124.21,122.78,135.31,154.77,171.27,186.75,201.85
3,4Yr,130.91,127.05,125.38,124.11,123.36,123.19,123.56,124.31,126.88,139.75,156.67,171.61,185.91,199.98
4,5Yr,122.93,120.31,119.38,118.91,119.02,119.77,121.10,122.89,127.81,140.63,156.15,170.34,184.15,197.80
5,6Yr,117.46,115.62,115.10,115.05,115.60,116.78,118.57,120.84,126.79,138.98,153.12,166.33,179.32,192.21
6,7Yr,112.15,111.14,111.04,111.38,112.28,113.78,115.83,118.34,124.68,136.03,148.82,160.95,172.96,184.90
7,8Yr,108.27,107.79,107.93,108.50,109.58,111.22,113.38,115.98,122.40,132.96,144.67,155.93,167.14,178.32
8,9Yr,104.69,104.71,105.09,105.85,107.09,108.84,111.07,113.70,120.07,129.93,140.74,151.23,161.73,172.22
9,10Yr,101.24,101.73,102.32,103.26,104.63,106.46,108.72,111.34,117.57,126.75,136.73,146.51,156.33,166.16


## Data processor

In [15]:
import QuantLib as ql
import pandas as pd

def build_ois_curve(valuation_date, ois_df):
    """Build OIS discount curve based on discount factors"""
    terms = ois_df['Term']
    discount_factors = ois_df['Discount']

    # Create calendar and date list
    calendar = ql.UnitedStates(ql.UnitedStates.NYSE)
    dates = [valuation_date]  # Start date is the valuation date
    adjusted_discount_factors = [1.0]  # Discount factor starts at 1.0

    # Iterate over all terms
    for i, term in enumerate(terms):
        if 'WK' in term:
            weeks = int(term.split(' ')[0])
            dates.append(calendar.advance(valuation_date, ql.Period(weeks, ql.Weeks)))
        elif 'MO' in term:
            months = int(term.split(' ')[0])
            dates.append(calendar.advance(valuation_date, ql.Period(months, ql.Months)))
        elif 'YR' in term or 'Y' in term:
            years = int(term.split(' ')[0])
            dates.append(calendar.advance(valuation_date, ql.Period(years, ql.Years)))
        adjusted_discount_factors.append(discount_factors.iloc[i])

    # Check if dates and discount factors are aligned
    if len(dates) != len(adjusted_discount_factors):
        raise ValueError(f"Mismatch between the number of dates and discount factors! Dates: {len(dates)}, Discount Factors: {len(adjusted_discount_factors)}")

    # Print debugging information
    print("Generated Dates and Discount Factors:")
    for date, factor in zip(dates, adjusted_discount_factors):
        print(f"Date: {date.ISO()}, Discount Factor: {factor:.6f}")

    # Use ActualActual.ISDA as the Day Counter
    day_counter = ql.Actual360()
    discount_curve = ql.DiscountCurve(dates, adjusted_discount_factors, day_counter)
    discount_curve.enableExtrapolation()
    return ql.YieldTermStructureHandle(discount_curve), discount_curve

def print_discount_curve(discount_curve, dates):
    """Print dates and discount factors for the discount curve"""
    print("\nDiscount Curve:")
    print("{:<15} {:<15}".format("Date", "Discount Factor"))
    print("-" * 30)
    for date in dates:
        discount_factor = discount_curve.discount(date)
        print("{:<15} {:<15.6f}".format(date.ISO(), discount_factor))

# Example usage
valuation_date = ql.Date(29, 12, 2023)
ql.Settings.instance().evaluationDate = valuation_date

# Call the function to build the OIS curve
discount_handle, discount_curve = build_ois_curve(valuation_date, ois_df)

# Print curve values
print_discount_curve(discount_curve, discount_curve.dates())

# Print original OIS DataFrame
print("\nOIS DataFrame at Build Step:")
print(ois_df)

def build_forward_curve(valuation_date, ois_df):
    """
    Build the forward curve using ois_df
    :param valuation_date: QuantLib.Date object, valuation date
    :param ois_df: Pandas DataFrame containing 'Term' and 'Market Rate' columns
    :return: (YieldTermStructureHandle, ForwardCurve)
    """
    calendar = ql.UnitedStates(ql.UnitedStates.NYSE)
    day_counter = ql.Actual360()

    # Extract terms and market rates from ois_df
    terms = ois_df['Term']
    market_rates = ois_df['Market Rate'] / 100  # Convert to decimal form

    # Create date and rate nodes
    dates = []
    rates = []

    print("Generating node dates and rates:")
    for i, term in enumerate(terms):
        if 'WK' in term:
            weeks = int(term.split(' ')[0])
            node_date = calendar.advance(valuation_date, ql.Period(weeks, ql.Weeks))
        elif 'MO' in term:
            months = int(term.split(' ')[0])
            node_date = calendar.advance(valuation_date, ql.Period(months, ql.Months))
        elif 'YR' in term or 'Y' in term:
            years = int(term.split(' ')[0])
            node_date = calendar.advance(valuation_date, ql.Period(years, ql.Years))
        else:
            raise ValueError(f"Unrecognized term format: {term}")

        # Add dates and corresponding rates
        if node_date > valuation_date:  # Ensure node date is valid
            dates.append(node_date)
            rates.append(market_rates.iloc[i])
            print(f"Term: {term}, Date: {node_date.ISO()}, Rate: {market_rates.iloc[i] * 100:.6f}%")

    # Ensure the number of node dates matches the number of rates
    if len(dates) != len(rates):
        raise RuntimeError("Mismatch between generated dates and rates, check the input data.")

    # Create forward curve based on rates
    forward_curve = ql.ForwardCurve(dates, rates, day_counter)

    return ql.YieldTermStructureHandle(forward_curve), forward_curve

def print_curve(forward_curve, start_date, months):
    """
    Print monthly rates for the curve
    :param forward_curve: Constructed forward curve object
    :param start_date: Start date for the curve
    :param months: Number of future months to print
    """
    print("\nGenerated Curve:")
    print("{:<15} {:<15}".format("Date", "Rate (%)"))
    print("-" * 30)
    calendar = ql.UnitedStates(ql.UnitedStates.NYSE)
    dates = [calendar.advance(start_date, ql.Period(i, ql.Months)) for i in range(months)]
    for date in dates:
        rate = forward_curve.forwardRate(date, date, ql.Actual360(), ql.Continuous).rate() * 100  # Convert to percentage
        print("{:<15} {:<15.6f}".format(date.ISO(), rate))

# Example usage
valuation_date = ql.Date(29, 12, 2023)
ql.Settings.instance().evaluationDate = valuation_date

# Build the forward curve
forward_handle, forward_curve = build_forward_curve(valuation_date, ois_df)

# Print monthly forward curve data (for the next 60 months)
print_curve(forward_curve, forward_curve.referenceDate(), 60)


Generated Dates and Discount Factors:
Date: 2023-12-29, Discount Factor: 1.000000
Date: 2024-01-05, Discount Factor: 0.998222
Date: 2024-01-12, Discount Factor: 0.997187
Date: 2024-01-19, Discount Factor: 0.996152
Date: 2024-01-29, Discount Factor: 0.994384
Date: 2024-02-29, Discount Factor: 0.990289
Date: 2024-04-01, Discount Factor: 0.985983
Date: 2024-04-29, Discount Factor: 0.981805
Date: 2024-05-29, Discount Factor: 0.977720
Date: 2024-07-01, Discount Factor: 0.973917
Date: 2024-07-29, Discount Factor: 0.969842
Date: 2024-08-29, Discount Factor: 0.966416
Date: 2024-09-30, Discount Factor: 0.962989
Date: 2024-10-29, Discount Factor: 0.959451
Date: 2024-11-29, Discount Factor: 0.956370
Date: 2024-12-30, Discount Factor: 0.953172
Date: 2025-06-30, Discount Factor: 0.936584
Date: 2025-12-29, Discount Factor: 0.921658
Date: 2026-12-29, Discount Factor: 0.893842
Date: 2027-12-29, Discount Factor: 0.866447
Date: 2028-12-29, Discount Factor: 0.838984
Date: 2029-12-31, Discount Factor: 0.8

In [16]:
import QuantLib as ql
import pandas as pd

# Prepare the Black volatility matrix
black_vol_data = pd.read_csv('Volatility_Black.csv')

# Replace 'Yr' with 'Y' to ensure compatibility with the QuantLib format
black_vol_data['Tenor'] = black_vol_data['Tenor'].str.replace('Yr', 'Y')

# Display the transformed Black volatility matrix
display(black_vol_data)


,Tenor,1.00%,1.50%,2.00%,2.50%,3.00%,3.50%,4.00%,5.00%,6.00%,7.00%,8.00%,9.00%,10.00%
0,1Y,79.00,62.88,51.91,43.60,36.91,31.35,26.66,19.12,17.49,20.51,22.61,24.27,25.69
1,2Y,79.46,63.76,53.63,46.33,40.72,36.17,32.29,25.63,25.16,28.35,30.37,31.75,32.82
2,3Y,76.34,61.55,52.26,45.75,40.91,37.13,34.02,29.12,29.14,31.37,32.77,33.74,34.49
3,4Y,72.57,58.93,50.51,44.71,40.47,37.24,34.67,30.92,30.99,32.45,33.34,33.95,34.43
4,5Y,68.84,56.18,48.49,43.26,39.50,36.69,34.51,31.51,31.55,32.56,33.18,33.63,34.01
5,6Y,66.03,54.05,46.83,41.97,38.51,35.95,34.00,31.40,31.29,31.94,32.32,32.62,32.89
6,7Y,63.32,52.02,45.25,40.70,37.48,35.11,33.31,30.94,30.66,31.02,31.20,31.35,31.51
7,8Y,61.29,50.45,43.97,39.64,36.58,34.33,32.63,30.38,29.96,30.09,30.13,30.18,30.26
8,9Y,59.38,48.98,42.78,38.65,35.72,33.56,31.94,29.77,29.24,29.19,29.11,29.08,29.10
9,10Y,57.52,47.54,41.62,37.66,34.85,32.79,31.22,29.11,28.46,28.27,28.10,28.00,27.97


In [17]:
import QuantLib as ql
import pandas as pd
from scipy.interpolate import interp1d
import numpy as np
from scipy.interpolate import CubicSpline

# Interpolation function: perform cubic spline interpolation when strike prices do not match
def get_interpolated_volatility(strikes, volatilities, target_strike):
    if len(strikes) != len(volatilities):
        min_len = min(len(strikes), len(volatilities))
        strikes = strikes[:min_len]
        volatilities = volatilities[:min_len]

    strikes = np.array(strikes)
    volatilities = np.array(volatilities)
    mask = ~np.isnan(volatilities)
    filtered_strikes = strikes[mask]
    filtered_volatilities = volatilities[mask]

    if len(filtered_strikes) < 2:
        return filtered_volatilities[0] if len(filtered_volatilities) > 0 else 0.0

    # Use cubic spline interpolation instead of linear interpolation
    interpolation = CubicSpline(filtered_strikes, filtered_volatilities, extrapolate=True)
    return float(interpolation(target_strike))

# Extract Black volatility surface data from black_vol_data
def get_vol_surface_data(black_vol_data):
    expiries = []
    strikes = []
    volatilities = []
    
    black_vol_data['Tenor'] = black_vol_data['Tenor'].astype(str).replace('Yr', 'Y').replace('yr', 'Y').str.strip()
    for tenor in black_vol_data['Tenor']:
        if 'Y' in tenor:
            expiries.append(ql.Period(int(tenor.replace('Y', '').strip()), ql.Years))
        elif 'M' in tenor:
            expiries.append(ql.Period(int(tenor.replace('M', '').strip()), ql.Months))
        else:
            raise ValueError(f"Unrecognized tenor format: {tenor}")
    
    strikes = [float(col.replace('%', '')) / 100 for col in black_vol_data.columns[1:]]
    volatilities = (black_vol_data.iloc[:, 1:].values / 100).tolist()
    
    return expiries, strikes, volatilities

# Create Black CapFloorTermVolSurface
def create_vol_surface_black(black_vol_data):
    expiries, strikes, black_volatility = get_vol_surface_data(black_vol_data)
    volatilities_handles = [
        [ql.QuoteHandle(ql.SimpleQuote(v)) for v in row]
        for row in black_volatility
    ]
    vol_surface = ql.CapFloorTermVolSurface(
        0,
        ql.UnitedStates(ql.UnitedStates.NYSE),
        ql.ModifiedFollowing,
        expiries,
        strikes,
        volatilities_handles,
        ql.Actual360()
    )
    return vol_surface, strikes, black_volatility

# Use Black model for Cap pricing
def price_cap_with_black_vol_surface(black_vol_data, discount_handle, ibor_leg, strike, schedule):
    vol_surface, strikes, volatilities_matrix = create_vol_surface_black(black_vol_data)
    
    if strike[0] in strikes:
        expiry_vols = [row[strikes.index(strike[0])] for row in volatilities_matrix]
    else:
        expiry_vols = [
            get_interpolated_volatility(strikes, row, strike[0]) 
            for row in volatilities_matrix
        ]
    interpolated_vol = get_interpolated_volatility(strikes, expiry_vols, strike[0])
    vol_handle = ql.QuoteHandle(ql.SimpleQuote(interpolated_vol))
    
    # Create Cap and set pricing engine
    cap = ql.Cap(ibor_leg, strike)
    engine_volatility = ql.BlackCapFloorEngine(discount_handle, vol_handle)
    cap.setPricingEngine(engine_volatility)
    
    cap_npv = cap.NPV()
    print(f"\nCap NPV with Black volatility surface: {cap_npv:.6f}")

    # Use Schedule to get accrual_start and accrual_end
    schedule_dates = schedule.dates()
    accrual_start = schedule_dates[:-1]
    accrual_end = schedule_dates[1:]

    # Print detailed cash flow information for each period
    print("\nOptionlets Data:")
    optionlets_prices = cap.optionletsPrice()
    optionlets_forward = cap.optionletsAtmForward()
    optionlets_discount_factors = cap.optionletsDiscountFactor()
    optionlets_std_dev = cap.optionletsStdDev()
    cap_rates = cap.capRates()
    
    for i in range(len(optionlets_prices)):
        print(
            f"Optionlet {i+1}: "
            f"Accrual Start: {accrual_start[i].ISO()}, "
            f"Accrual End: {accrual_end[i].ISO()}, "
            f"Price: {optionlets_prices[i]:.6f}, "
            f"ATM Forward: {optionlets_forward[i]:.6f}, "
            f"Discount Factor: {optionlets_discount_factors[i]:.6f}, "
            f"Standard Deviation: {optionlets_std_dev[i]:.6f}, "
            f"Cap Rate: {cap_rates[i]:.6f}"
        )

    return cap_npv

# Set the valuation date to December 29, 2023
valuation_date = ql.Date(29, 12, 2023)
ql.Settings.instance().evaluationDate = valuation_date

# Build the OIS discount curve
discount_handle, discount_curve = build_ois_curve(valuation_date, ois_df)

# Build the forward curve
forward_handle, forward_curve = build_forward_curve(valuation_date, ois_df)

# Define the start and end dates for the Cap
start_date = ql.Date(3, 6, 2024)
end_date = ql.Date(3, 6, 2029)

# Use Schedule to generate payment dates
schedule = ql.Schedule(
    start_date,
    end_date,
    ql.Period(ql.Quarterly),
    ql.UnitedStates(ql.UnitedStates.NYSE),
    ql.ModifiedFollowing,
    ql.ModifiedFollowing,
    ql.DateGeneration.Backward,
    False
)

# Create Forward Index based on Market Rate
custom_forward_index = ql.IborIndex(
    "MyForwardIndex",
    ql.Period("3M"),
    0,
    ql.USDCurrency(),
    ql.UnitedStates(ql.UnitedStates.NYSE),
    ql.ModifiedFollowing,
    False,
    ql.Actual360(),
    forward_handle  # Use forward curve
)

# Fill in Fixings using SOFR data
calendar = ql.UnitedStates(ql.UnitedStates.NYSE)
for index, row in sofr_df.iterrows():
    date = ql.Date(row['Date'].day, row['Date'].month, row['Date'].year)
    if calendar.isBusinessDay(date) and date <= valuation_date:
        rate = row['SOFR']
        custom_forward_index.addFixing(date, rate)

# Create IborLeg
ibor_leg_forward = ql.IborLeg([10e6], schedule, custom_forward_index)
strike = [0.029115]

# Use Black volatility surface for pricing
cap_npv = price_cap_with_black_vol_surface(black_vol_data, discount_handle, ibor_leg_forward, strike, schedule)

# Print payment dates
schedule_dates = schedule.dates()
print("\nOriginal Schedule Dates:")
print([d.ISO() for d in schedule_dates])


Generated Dates and Discount Factors:
Date: 2023-12-29, Discount Factor: 1.000000
Date: 2024-01-05, Discount Factor: 0.998222
Date: 2024-01-12, Discount Factor: 0.997187
Date: 2024-01-19, Discount Factor: 0.996152
Date: 2024-01-29, Discount Factor: 0.994384
Date: 2024-02-29, Discount Factor: 0.990289
Date: 2024-04-01, Discount Factor: 0.985983
Date: 2024-04-29, Discount Factor: 0.981805
Date: 2024-05-29, Discount Factor: 0.977720
Date: 2024-07-01, Discount Factor: 0.973917
Date: 2024-07-29, Discount Factor: 0.969842
Date: 2024-08-29, Discount Factor: 0.966416
Date: 2024-09-30, Discount Factor: 0.962989
Date: 2024-10-29, Discount Factor: 0.959451
Date: 2024-11-29, Discount Factor: 0.956370
Date: 2024-12-30, Discount Factor: 0.953172
Date: 2025-06-30, Discount Factor: 0.936584
Date: 2025-12-29, Discount Factor: 0.921658
Date: 2026-12-29, Discount Factor: 0.893842
Date: 2027-12-29, Discount Factor: 0.866447
Date: 2028-12-29, Discount Factor: 0.838984
Date: 2029-12-31, Discount Factor: 0.8